# Character-Level Transformer (Java autocomplete)

In this notebook, I try to make a small mini-GPT - a simple version of the kind of model used in ChatGPT or GitHub Copilot.

When I say "character-level" I mean the model gets characters, not tokens. For example:
- input: `public void get`
- model should predict next char: `N` (so it can become `getName`)

It is easier than subword tokenization, but idea is same.

---

## Step 0 — Setup & imports

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import math
import os
from pathlib import Path

# I check which device is available
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Silicon GPU
    print("Device: Apple Silicon GPU (MPS) — I use it")
elif torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA GPU (for Colab)
    print("Device: CUDA GPU — I use it")
else:
    device = torch.device("cpu")
    print("Device: CPU")

print(f"PyTorch version: {torch.__version__}")

Device: Apple Silicon GPU (MPS) — I use it
PyTorch version: 2.8.0


# Part 1: Character-Level Transformer for Java Autocomplete

## Lecture Notes: Building a Mini Language Model from Scratch

---

## What Are We Building?

The goal of this notebook is to build a **mini-GPT** — a small-scale version of the same architecture that powers ChatGPT, GitHub Copilot, and other modern language models.

**Character-level** means the model will:
- Take a sequence of characters as input: `public void get`
- Predict the next character: `N` (→ completing to `getName`)

This approach is simpler than working with tokens (words or subwords), but the underlying principles are exactly the same. Once this is understood, scaling up to production-level models becomes much more intuitive.

### Why Start With Character-Level?

Working at the character level has pedagogical advantages:
- **Smaller vocabulary**: Instead of 50,000+ tokens, there are only ~100 unique characters
- **Easier to debug**: The model's behavior is more transparent
- **Same architecture**: The Transformer architecture is identical to production models

---

## Step 0: Imports and Setup

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import math
import os
from pathlib import Path

# Check available compute devices
# Modern deep learning benefits enormously from GPU acceleration
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple Silicon GPU
    print("Using Apple Silicon GPU (MPS)")
elif torch.cuda.is_available():
    device = torch.device("cuda")  # NVIDIA GPU (for Colab or workstations)
    print("Using CUDA GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")

print(f"PyTorch version: {torch.__version__}")

Using Apple Silicon GPU (MPS)
PyTorch version: 2.8.0


---
## Step 1: Data Preparation

### Background: How Does a Neural Network "See" Text?

Neural networks operate exclusively on numbers — they cannot directly process characters, words, or any symbolic data. This means every piece of text must be converted into numerical form before the model can work with it.

The conversion process involves three steps:

1. **Build a vocabulary** — create a mapping from each unique character to a number (an index)
2. **Encode** — convert text into a sequence of these indices
3. **Decode** — convert indices back to text for human-readable output

**Concrete Example:**
```
Vocabulary: {'p': 0, 'u': 1, 'b': 2, 'l': 3, 'i': 4, 'c': 5, ' ': 6, ...}
Text:       "public"
Encoded:    [0, 1, 2, 3, 4, 5]
```

### Quick Math Refresher: One-Hot Encoding

Later, these indices get converted into **one-hot vectors** or looked up in an **embedding table**. A one-hot vector has all zeros except for a single 1 at the index position:

- If vocabulary size = 10 and character index = 3:
- One-hot: `[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`

In practice, models use **embeddings** instead — dense vectors that are learned during training. More on this later.

In [43]:
# Starting with a small Java code sample for quick experimentation
# Later, this can be replaced with a real dataset of thousands of files

SAMPLE_JAVA_CODE = '''
public class HelloWorld {
    public static void main(String[] args) {
        System.out.println("Hello, World!");
    }
}

public class Calculator {
    private int value;
    
    public Calculator() {
        this.value = 0;
    }
    
    public int add(int x) {
        this.value += x;
        return this.value;
    }
    
    public int subtract(int x) {
        this.value -= x;
        return this.value;
    }
    
    public int getValue() {
        return this.value;
    }
    
    public void setValue(int value) {
        this.value = value;
    }
}

public class StringUtils {
    public static String reverse(String s) {
        StringBuilder sb = new StringBuilder(s);
        return sb.reverse().toString();
    }
    
    public static boolean isEmpty(String s) {
        return s == null || s.length() == 0;
    }
    
    public static String capitalize(String s) {
        if (isEmpty(s)) {
            return s;
        }
        return Character.toUpperCase(s.charAt(0)) + s.substring(1);
    }
}
'''

print(f"Sample size: {len(SAMPLE_JAVA_CODE)} characters")

Sample size: 1025 characters


In [44]:
class CharTokenizer:
    """
    A simple character-level tokenizer.
    
    This class builds a vocabulary from all unique characters in the text
    and provides methods to convert between text and numerical indices.
    
    Why "tokenizer"? In NLP, the process of breaking text into units is called
    tokenization. Here, each token is a single character.
    """
    
    def __init__(self, text: str):
        # Collect all unique characters and sort them for reproducibility
        # Sorting ensures the same text always produces the same vocabulary
        chars = sorted(list(set(text)))
        self.vocab_size = len(chars)
        
        # Create bidirectional mappings
        # char_to_idx: 'a' → 0, 'b' → 1, etc.
        # idx_to_char: 0 → 'a', 1 → 'b', etc.
        self.char_to_idx = {ch: i for i, ch in enumerate(chars)}
        self.idx_to_char = {i: ch for i, ch in enumerate(chars)}
        
        print(f"Vocabulary created: {self.vocab_size} unique characters")
        print(f"Examples: {list(self.char_to_idx.items())[:10]}...")
    
    def encode(self, text: str) -> list[int]:
        """Convert text to a list of integers (indices)"""
        return [self.char_to_idx[ch] for ch in text]
    
    def decode(self, indices: list[int]) -> str:
        """Convert a list of integers back to text"""
        return ''.join([self.idx_to_char[i] for i in indices])


# Create the tokenizer
tokenizer = CharTokenizer(SAMPLE_JAVA_CODE)

# Verify that encode/decode are inverse operations (roundtrip test)
test_text = "public void"
encoded = tokenizer.encode(test_text)
decoded = tokenizer.decode(encoded)
print(f"\nRoundtrip test: '{test_text}' → {encoded} → '{decoded}'")
assert test_text == decoded, "Tokenizer error: roundtrip failed!"

Vocabulary created: 51 unique characters
Examples: [('\n', 0), (' ', 1), ('!', 2), ('"', 3), ('(', 4), (')', 5), ('+', 6), (',', 7), ('-', 8), ('.', 9)]...

Roundtrip test: 'public void' → [38, 42, 26, 34, 33, 27, 1, 43, 37, 33, 28] → 'public void'


### Background: How Are Training Examples Formed?

A language model learns to predict the **next token** given all previous tokens. This is sometimes called **autoregressive** modeling — the model's output at step N becomes part of the input for step N+1.

From the text `"public class"`, the following training pairs (input → target) are created:

```
"p"           → "u"        (given 'p', predict 'u')
"pu"          → "b"        (given 'pu', predict 'b')
"pub"         → "l"        (given 'pub', predict 'l')
"publ"        → "i"        
"publi"       → "c"        
"public"      → " "        (the space character!)
"public "     → "c"        
"public c"    → "l"        
...
```

### Practical Implementation: Fixed Context Windows

In practice, using variable-length inputs would be inefficient. Instead, a fixed **context length** (also called sequence length or block size) is used. All inputs are padded or truncated to this length.

The clever trick: from one context window, multiple predictions can be made simultaneously. If the context length is 8, then from positions 0-7, the model predicts positions 1-8. This is much more efficient than making one prediction at a time.

### Math Refresher: Tensors and Shapes

Throughout this notebook, tensor shapes are annotated as `(B, T, C)` where:
- **B** = Batch size (number of examples processed in parallel)
- **T** = Time steps / Sequence length (number of tokens)
- **C** = Channels / Features (dimension of each token's representation)

This notation comes from signal processing and is standard in deep learning.

In [45]:
class CharDataset(Dataset):
    """
    PyTorch Dataset for training a language model.
    
    Each example consists of:
    - x: a sequence of context_length characters (the input)
    - y: the same sequence shifted by 1 position (the target)
    
    Example with context_length=8:
    - x: "public v"  [characters at positions 0-7]
    - y: "ublic vo"  [characters at positions 1-8]
    
    The model learns:
    - From x[0] alone, predict y[0]
    - From x[0:2], predict y[1]
    - From x[0:3], predict y[2]
    - And so on...
    
    This is efficient because one forward pass produces context_length predictions.
    """
    
    def __init__(self, text: str, tokenizer: CharTokenizer, context_length: int):
        self.context_length = context_length
        # Convert entire text to a tensor of indices
        # dtype=torch.long because indices must be integers
        self.data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
        print(f"Dataset created: {len(self.data)} tokens, {len(self)} training examples")
    
    def __len__(self):
        # Number of possible windows that can be extracted
        # Subtract context_length because the last window needs one extra token for the target
        return len(self.data) - self.context_length
    
    def __getitem__(self, idx):
        # Extract a window of [idx : idx + context_length + 1]
        # Then split into input (x) and target (y)
        chunk = self.data[idx : idx + self.context_length + 1]
        x = chunk[:-1]  # Everything except the last token
        y = chunk[1:]   # Everything except the first token
        return x, y


# Create the dataset with a modest context length to start
CONTEXT_LENGTH = 64  # The model will "see" up to 64 characters at a time
dataset = CharDataset(SAMPLE_JAVA_CODE, tokenizer, CONTEXT_LENGTH)

# Examine one training example
x, y = dataset[0]
print(f"\nExample training pair:")
print(f"x (input):  '{tokenizer.decode(x.tolist())}'")
print(f"y (target): '{tokenizer.decode(y.tolist())}'")
print(f"\nTensor shapes: x={x.shape}, y={y.shape}")

Dataset created: 1025 tokens, 961 training examples

Example training pair:
x (input):  '
public class HelloWorld {
    public static void main(String[] '
y (target): 'public class HelloWorld {
    public static void main(String[] a'

Tensor shapes: x=torch.Size([64]), y=torch.Size([64])


---
## Step 2: The Transformer Architecture

### Background: Why Transformers?

Before 2017, sequence modeling (text, speech, time series) was dominated by **Recurrent Neural Networks (RNNs)** and their variants like **LSTMs** (Long Short-Term Memory). These had significant limitations:

1. **Sequential processing**: RNNs must process tokens one at a time, making parallelization impossible. This means slow training and inference.

2. **Vanishing gradients**: Information from early tokens gets "diluted" as it passes through many steps. Learning long-range dependencies (like matching an opening brace with its closing brace 100 characters later) becomes very difficult.

The **Transformer** architecture, introduced in the paper "Attention Is All You Need" (2017), solves both problems through a mechanism called **Self-Attention**.

---

### Core Concept: What is Self-Attention?

**The intuition:** Each token "looks at" all other tokens (or in our case, all *previous* tokens) and decides how much attention to pay to each one.

**Concrete example:** In the code `this.value = x;`, when the model reaches `x`, it should "attend to":
- `value` — to understand what's being assigned
- `this` — to understand the context (it's inside a class method)
- `.` and `=` — to understand the syntactic structure

### The Query-Key-Value Mechanism

Each token creates three different vectors:

1. **Query (Q)**: "What am I looking for?"
   - Think of this as a question the token is asking
   
2. **Key (K)**: "What do I contain?"
   - Think of this as a label or tag describing the token's content
   
3. **Value (V)**: "What information do I provide if someone attends to me?"
   - This is the actual information that gets passed along

### The Attention Formula

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V$$

Let's break this down step by step:

1. **$QK^T$** — Compute "similarity" between each query and all keys
   - Matrix multiplication: if Q is (T, d) and K is (T, d), then QK^T is (T, T)
   - Entry (i, j) tells us how much token i's query matches token j's key

2. **$\div \sqrt{d_k}$** — Scale by the square root of the key dimension
   - **Why?** Without scaling, the dot products can become very large when d_k is big
   - Large values push softmax into regions with tiny gradients (saturation)
   - Dividing by $\sqrt{d_k}$ keeps values in a reasonable range

3. **softmax** — Convert scores to probabilities (sum to 1)
   - Each row becomes a probability distribution over all tokens
   - Higher scores → higher probabilities

4. **$\times V$** — Weighted sum of values
   - Each token's output is a mixture of all values, weighted by attention

### Math Refresher: Softmax

Softmax converts a vector of arbitrary real numbers into a probability distribution:

$$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}$$

Properties:
- All outputs are positive (due to exponential)
- All outputs sum to 1
- Larger inputs get larger probabilities
- The exponential makes it "winner-take-more" — differences are amplified

In [46]:
class SelfAttention(nn.Module):
    """
    A single "head" of self-attention.
    
    Parameters:
    - embed_dim: the dimension of token embeddings (input size)
    - head_dim: the dimension of Q, K, V vectors in this head
    - context_length: maximum sequence length (needed for the causal mask)
    
    Note: In multi-head attention, head_dim is typically embed_dim // num_heads,
    so all heads together have the same total dimensionality as embed_dim.
    """
    
    def __init__(self, embed_dim: int, head_dim: int, context_length: int):
        super().__init__()
        
        # Linear projections to create Q, K, V from input embeddings
        # These are learned transformations: input → query/key/value space
        # bias=False is common in attention (the original paper used no bias)
        self.query = nn.Linear(embed_dim, head_dim, bias=False)
        self.key = nn.Linear(embed_dim, head_dim, bias=False)
        self.value = nn.Linear(embed_dim, head_dim, bias=False)
        
        # Causal mask: prevents attending to future tokens
        # This is essential for autoregressive generation
        # A lower triangular matrix: position i can only see positions 0..i
        #
        # Example for 4 positions:
        # [[1, 0, 0, 0],   Position 0 sees only itself
        #  [1, 1, 0, 0],   Position 1 sees positions 0, 1
        #  [1, 1, 1, 0],   Position 2 sees positions 0, 1, 2
        #  [1, 1, 1, 1]]   Position 3 sees all positions
        mask = torch.tril(torch.ones(context_length, context_length))
        self.register_buffer('mask', mask)  # register_buffer: saved with model but not trained
        
        # Scale factor: 1/sqrt(d_k) as in the attention formula
        self.scale = head_dim ** -0.5
    
    def forward(self, x):
        B, T, C = x.shape  # Batch, Time (sequence length), Channels (embed_dim)
        
        # Project input into query, key, value spaces
        q = self.query(x)  # (B, T, head_dim)
        k = self.key(x)    # (B, T, head_dim)
        v = self.value(x)  # (B, T, head_dim)
        
        # Compute attention scores: how much each token attends to each other token
        # q @ k.transpose(-2, -1) produces a (T, T) matrix per batch element
        # Entry (i, j) = dot product of query_i and key_j = "relevance" of j to i
        scores = (q @ k.transpose(-2, -1)) * self.scale  # (B, T, T)
        
        # Apply causal mask: set "future" positions to -infinity
        # After softmax, -inf becomes 0 (e^(-inf) = 0)
        # This ensures each position only attends to itself and past positions
        scores = scores.masked_fill(self.mask[:T, :T] == 0, float('-inf'))
        
        # Softmax along last dimension: each row becomes a probability distribution
        attn_weights = F.softmax(scores, dim=-1)  # (B, T, T)
        
        # Weighted sum of values: each output is a mixture of all values
        # weighted by the attention probabilities
        out = attn_weights @ v  # (B, T, head_dim)
        
        return out


# Visualize the causal mask structure
print("Causal mask (8x8 example):")
print("1 = can see, 0 = cannot see (future)")
print(torch.tril(torch.ones(8, 8)))

Causal mask (8x8 example):
1 = can see, 0 = cannot see (future)
tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])


### Multi-Head Attention: Why Multiple Heads?

A single attention head can only focus on one type of relationship at a time. **Multi-head attention** runs several attention mechanisms in parallel, each with its own Q, K, V projections.

**Intuition:** Different heads learn to focus on different aspects:
- Head 1: Might track syntactic structure (braces, semicolons, parentheses)
- Head 2: Might track data types (int, String, boolean)
- Head 3: Might track variable names and their usage
- Head 4: Might track control flow (if, else, return)

The outputs of all heads are concatenated and then projected back to the original dimension.

### Math Details

If there are $h$ heads and the total embedding dimension is $d_{model}$:
- Each head has dimension $d_k = d_{model} / h$
- After concatenation: $h \times d_k = d_{model}$ (back to original size)
- A final linear layer projects this concatenation

This design keeps the computational cost similar to single-head attention of the same dimension, while allowing the model to attend to information from different representation subspaces.

In [47]:
class MultiHeadAttention(nn.Module):
    """
    Multi-Head Self-Attention.
    
    Runs multiple SelfAttention heads in parallel,
    concatenates their outputs, and projects back to embed_dim.
    
    This allows the model to jointly attend to information
    from different representation subspaces at different positions.
    """
    
    def __init__(self, embed_dim: int, num_heads: int, context_length: int):
        super().__init__()
        
        # embed_dim must be divisible by num_heads so each head gets equal dimensions
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"
        
        head_dim = embed_dim // num_heads
        
        # Create multiple attention heads
        # nn.ModuleList ensures PyTorch tracks these as submodules
        self.heads = nn.ModuleList([
            SelfAttention(embed_dim, head_dim, context_length)
            for _ in range(num_heads)
        ])
        
        # Final projection after concatenating all heads
        # This allows the model to mix information across heads
        self.proj = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        # Run all heads and concatenate their outputs
        head_outputs = [head(x) for head in self.heads]
        concat = torch.cat(head_outputs, dim=-1)  # (B, T, embed_dim)
        
        # Project concatenated output
        out = self.proj(concat)
        return out

### Feed-Forward Network (FFN): Adding Non-linearity

After attention, each position passes through a simple two-layer neural network:

$$\text{FFN}(x) = \text{GELU}(xW_1 + b_1)W_2 + b_2$$

**Why is this needed?**

Attention only "mixes" information between positions — it computes weighted averages. This is fundamentally a linear operation (weighted sums).

The FFN adds **non-linear transformation** at each position independently. This is where much of the model's "reasoning" capacity comes from.

**Architecture details:**
- First layer expands dimension (typically 4× the embedding dimension)
- GELU activation provides non-linearity
- Second layer projects back to original dimension

### Math Refresher: GELU (Gaussian Error Linear Unit)

GELU is a smooth activation function, similar to ReLU but differentiable everywhere:

$$\text{GELU}(x) = x \cdot \Phi(x)$$

where $\Phi(x)$ is the cumulative distribution function of the standard normal distribution.

**Intuition:** GELU smoothly gates values based on how "likely" they are to be positive. Unlike ReLU (which has a hard cutoff at 0), GELU has a smooth transition, which often leads to better gradient flow during training.

In [48]:
class FeedForward(nn.Module):
    """
    Position-wise Feed-Forward Network.
    
    Structure: Linear → GELU → Linear
    
    The inner dimension (ff_dim) is typically 4x the embedding dimension.
    This "expand then contract" pattern is common in transformers.
    """
    
    def __init__(self, embed_dim: int, ff_dim: int = None):
        super().__init__()
        
        if ff_dim is None:
            ff_dim = 4 * embed_dim  # Standard ratio from the original Transformer paper
        
        self.net = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.GELU(),  # Smoother than ReLU, works better for transformers
            nn.Linear(ff_dim, embed_dim),
        )
    
    def forward(self, x):
        return self.net(x)

### The Transformer Block: Putting It Together

One transformer block combines attention and FFN with two critical additions:

```
x → LayerNorm → MultiHeadAttention → + (residual) → LayerNorm → FFN → + (residual) → out
    └──────────────────────────────────────────────┘            └───────────────────────┘
```

### Residual Connections (Skip Connections)

The `+` signs indicate **residual connections**: adding the input directly to the output.

```python
out = x + layer(x)  # Instead of just: out = layer(x)
```

**Why residual connections?**

1. **Gradient flow**: In deep networks, gradients can vanish (approach zero) as they propagate backward through many layers. Residual connections provide a "highway" for gradients to flow directly.

2. **Easier optimization**: The layer only needs to learn the "residual" — what to add to the input — rather than the full transformation. This is often easier.

3. **Identity initialization**: At the start of training, if layer outputs are near zero, the block acts like an identity function. This makes training more stable.

### Layer Normalization

Layer normalization normalizes activations across the feature dimension:

$$\text{LayerNorm}(x) = \gamma \cdot \frac{x - \mu}{\sigma + \epsilon} + \beta$$

where $\mu$ and $\sigma$ are the mean and standard deviation computed over the features, and $\gamma$, $\beta$ are learned parameters.

**Why normalize?**
- Keeps activations in a reasonable range
- Reduces internal covariate shift (changes in input distribution during training)
- Makes training more stable and often faster

**Pre-LN vs Post-LN:** GPT uses "Pre-LN" — normalization *before* each sublayer. This tends to be more stable for training deep networks.

In [49]:
class TransformerBlock(nn.Module):
    """
    A single Transformer block: Attention + FFN with residual connections.
    
    Uses Pre-LN architecture (normalization before each sublayer),
    which is the standard for GPT-style models.
    """
    
    def __init__(self, embed_dim: int, num_heads: int, context_length: int):
        super().__init__()
        
        self.ln1 = nn.LayerNorm(embed_dim)  # Normalization before attention
        self.attn = MultiHeadAttention(embed_dim, num_heads, context_length)
        self.ln2 = nn.LayerNorm(embed_dim)  # Normalization before FFN
        self.ffn = FeedForward(embed_dim)
    
    def forward(self, x):
        # Pre-LN architecture (as used in GPT-2/3)
        # Note: residual connection adds input to output of each sublayer
        x = x + self.attn(self.ln1(x))  # Attention with residual
        x = x + self.ffn(self.ln2(x))   # FFN with residual
        return x

### Positional Encoding: Teaching Order to Attention

**The problem:** Self-attention by itself has no notion of position or order!

For attention, `"public class"` and `"class public"` produce the same output (just reordered). This is because attention only computes pairwise similarities — it doesn't know which token came first.

**The solution:** Add positional information to the token embeddings.

In GPT-style models, this is done with **learned positional embeddings** — a lookup table where each position (0, 1, 2, ...) has its own learned vector:

```
Position 0 → vector [0.1, -0.3, 0.2, ...]
Position 1 → vector [0.2, 0.5, -0.1, ...]
Position 2 → vector [-0.1, 0.4, 0.3, ...]
...
```

These vectors are added to the token embeddings:

$$\text{input} = \text{TokenEmbedding}(\text{token}) + \text{PositionEmbedding}(\text{position})$$

The model learns what position means during training — for example, that position 0 is often the start of a statement, or that certain syntax patterns appear at specific relative positions.

**Alternative: Sinusoidal encodings** (from the original Transformer paper) use fixed sine and cosine functions. Learned embeddings are more flexible and are the standard choice for GPT models.

---
## Step 3: Assembling the Complete Model

Now all the pieces come together into the complete mini-GPT architecture:

1. **Token Embedding**: Convert character indices to dense vectors
2. **Position Embedding**: Add positional information  
3. **N × Transformer Blocks**: The core processing
4. **Final LayerNorm**: Stabilize outputs
5. **Output Projection**: Convert vectors to logits over vocabulary

### Weight Initialization

Proper initialization is crucial for training stability. The standard approach for transformers:
- Linear layers: Normal distribution with std=0.02
- Embeddings: Normal distribution with std=0.02
- Biases: Initialize to zero

This keeps initial activations small but not zero, allowing gradients to flow.

In [50]:
class MiniGPT(nn.Module):
    """
    A miniature GPT model for text generation.
    
    Architecture:
    1. Token Embedding: character → vector
    2. Position Embedding: position → vector  
    3. N × Transformer Blocks
    4. Final LayerNorm
    5. Output projection: vector → logits for each character in vocabulary
    
    The output logits can be converted to probabilities with softmax.
    """
    
    def __init__(
        self,
        vocab_size: int,
        embed_dim: int = 128,
        num_heads: int = 4,
        num_layers: int = 4,
        context_length: int = 64,
    ):
        super().__init__()
        
        self.context_length = context_length
        
        # Embedding layers
        # Token embedding: maps each vocabulary index to a dense vector
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)
        # Position embedding: maps each position (0 to context_length-1) to a dense vector
        self.position_embedding = nn.Embedding(context_length, embed_dim)
        
        # Stack of transformer blocks
        # nn.Sequential applies them in order
        self.blocks = nn.Sequential(*[
            TransformerBlock(embed_dim, num_heads, context_length)
            for _ in range(num_layers)
        ])
        
        # Final layer normalization and projection to vocabulary
        self.ln_final = nn.LayerNorm(embed_dim)
        self.output_proj = nn.Linear(embed_dim, vocab_size)
        
        # Initialize weights (important for stable training)
        self.apply(self._init_weights)
        
        # Count and display parameters
        n_params = sum(p.numel() for p in self.parameters())
        print(f"Model created: {n_params:,} parameters ({n_params/1e6:.2f}M)")
    
    def _init_weights(self, module):
        """Standard initialization for transformer models"""
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx):
        """
        Forward pass of the model.
        
        Args:
            idx: (B, T) tensor of token indices
            
        Returns:
            (B, T, vocab_size) tensor of logits for each position
        """
        B, T = idx.shape
        
        # Get token embeddings: look up each index in the embedding table
        tok_emb = self.token_embedding(idx)  # (B, T, embed_dim)
        
        # Get position embeddings: create position indices [0, 1, 2, ..., T-1]
        pos = torch.arange(T, device=idx.device)  # (T,)
        pos_emb = self.position_embedding(pos)  # (T, embed_dim)
        
        # Add embeddings together
        # Broadcasting: pos_emb (T, embed_dim) is broadcast across batch dimension
        x = tok_emb + pos_emb  # (B, T, embed_dim)
        
        # Pass through transformer blocks
        x = self.blocks(x)
        x = self.ln_final(x)
        
        # Project to vocabulary size to get logits
        # Logits are unnormalized scores; apply softmax to get probabilities
        logits = self.output_proj(x)  # (B, T, vocab_size)
        
        return logits
    
    @torch.no_grad()  # Disable gradient computation for inference
    def generate(self, idx, max_new_tokens: int, temperature: float = 1.0):
        """
        Generate text autoregressively.
        
        Args:
            idx: (B, T) tensor of starting context token indices
            max_new_tokens: number of tokens to generate
            temperature: controls randomness
                - temperature > 1.0: more random (flatter distribution)
                - temperature < 1.0: more deterministic (sharper distribution)
                - temperature = 1.0: use raw model probabilities
        
        Returns:
            (B, T + max_new_tokens) tensor with generated tokens appended
        """
        for _ in range(max_new_tokens):
            # Crop context if it's longer than context_length
            # The model can only see context_length tokens at a time
            idx_cond = idx[:, -self.context_length:]
            
            # Get model predictions
            logits = self(idx_cond)  # (B, T, vocab_size)
            
            # Focus on the last position only (next token prediction)
            logits = logits[:, -1, :] / temperature  # (B, vocab_size)
            
            # Convert logits to probabilities
            probs = F.softmax(logits, dim=-1)
            
            # Sample from the probability distribution
            # multinomial: randomly selects indices according to probabilities
            next_token = torch.multinomial(probs, num_samples=1)  # (B, 1)
            
            # Append to the sequence
            idx = torch.cat([idx, next_token], dim=1)  # (B, T+1)
        
        return idx


# Create the model
model = MiniGPT(
    vocab_size=tokenizer.vocab_size,
    embed_dim=128,      # Size of token vectors
    num_heads=4,        # Number of attention heads
    num_layers=4,       # Number of transformer blocks
    context_length=CONTEXT_LENGTH,
).to(device)

# Verify that forward pass works with a test input
test_input = torch.randint(0, tokenizer.vocab_size, (2, 32)).to(device)
test_output = model(test_input)
print(f"\nForward pass test: input {test_input.shape} → output {test_output.shape}")

Model created: 813,107 parameters (0.81M)

Forward pass test: input torch.Size([2, 32]) → output torch.Size([2, 32, 51])


---
## Step 4: Training

### Background: The Loss Function

Training uses **Cross-Entropy Loss** — the standard choice for classification problems.

For each position in the sequence, the model outputs probabilities for all possible next characters. The loss measures how well the model's predicted probabilities match reality (where the "correct" character should have probability 1 and all others should have probability 0).

$$\mathcal{L} = -\frac{1}{N}\sum_{i} \log P(y_i | x_{<i})$$

**Breaking this down:**

- $P(y_i | x_{<i})$: The model's predicted probability for the correct next character $y_i$, given all previous characters $x_{<i}$
- $\log P$: Taking the logarithm (base e, i.e., natural log)
- $-\log P$: Negative log (since $0 < P ≤ 1$, $\log P$ is negative or zero; negating makes it positive)
- $\sum_i$: Sum over all positions
- $\frac{1}{N}$: Average over all positions

**Intuition:**
- If the model assigns high probability to the correct character → $-\log P$ is small → low loss
- If the model assigns low probability to the correct character → $-\log P$ is large → high loss

### Math Refresher: Why Log-Probability?

Using logarithms has several benefits:
1. **Numerical stability**: Probabilities can be very small (like 0.0001); logs prevent underflow
2. **Additive property**: $\log(P_1 \times P_2) = \log P_1 + \log P_2$, making joint probabilities easier to work with
3. **Information theory**: $-\log P$ is the number of bits needed to encode an event with probability $P$

### Training Mechanics

The training loop follows the standard pattern:
1. **Forward pass**: Compute predictions
2. **Compute loss**: Compare predictions to targets
3. **Backward pass**: Compute gradients (automatic differentiation)
4. **Update weights**: Use optimizer to adjust parameters

In [51]:
def train_model(
    model,
    dataset,
    tokenizer,
    epochs: int = 100,
    batch_size: int = 32,
    learning_rate: float = 3e-4,
    print_every: int = 10,
):
    """
    Training loop for the language model.
    
    Args:
        model: The MiniGPT model to train
        dataset: CharDataset containing training examples
        tokenizer: CharTokenizer for encoding/decoding text
        epochs: Number of complete passes through the dataset
        batch_size: Number of examples per gradient update
        learning_rate: Step size for optimization (3e-4 is a common default for transformers)
        print_every: Print progress every N epochs
    """
    
    # DataLoader handles batching and shuffling
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # AdamW is the standard optimizer for transformers
    # It's Adam with decoupled weight decay, which works better for this architecture
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    
    # Set model to training mode (enables dropout, etc. if present)
    model.train()
    
    for epoch in range(epochs):
        total_loss = 0
        num_batches = 0
        
        for x, y in dataloader:
            # Move data to the same device as the model
            x, y = x.to(device), y.to(device)
            
            # Forward pass: compute predictions
            logits = model(x)  # (B, T, vocab_size)
            
            # Reshape for cross_entropy:
            # - Predictions: (B*T, vocab_size) — one prediction per position
            # - Targets: (B*T,) — one target index per position
            B, T, V = logits.shape
            loss = F.cross_entropy(
                logits.view(B * T, V),
                y.view(B * T)
            )
            
            # Backward pass: compute gradients
            optimizer.zero_grad()  # Clear gradients from previous step
            loss.backward()        # Compute gradients via backpropagation
            optimizer.step()       # Update weights using gradients
            
            total_loss += loss.item()
            num_batches += 1
        
        avg_loss = total_loss / num_batches
        
        # Print progress and generate a sample
        if (epoch + 1) % print_every == 0:
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
            
            # Generate a sample to see qualitative progress
            model.eval()  # Switch to evaluation mode
            prompt = "public "
            prompt_ids = torch.tensor([tokenizer.encode(prompt)]).to(device)
            generated = model.generate(prompt_ids, max_new_tokens=50, temperature=0.8)
            generated_text = tokenizer.decode(generated[0].tolist())
            print(f"  Sample: {generated_text[:100]}...")
            model.train()  # Switch back to training mode
    
    return model

In [52]:
# Train the model
print("Starting training...\n")
model = train_model(
    model,
    dataset,
    tokenizer,
    epochs=100,           # Number of passes through the data
    batch_size=16,        # Examples per gradient update
    learning_rate=3e-4,   # Step size (3e-4 = 0.0003)
    print_every=20,       # Print every 20 epochs
)

Starting training...

Epoch 20/100, Loss: 0.1015
  Sample: public void setValue(int value) {
        this.value = va...
Epoch 40/100, Loss: 0.0873
  Sample: public static String capitalize(String s) {
        if (i...
Epoch 60/100, Loss: 0.0761
  Sample: public Calculator() {
        this.value = 0;
    }
    
...
Epoch 80/100, Loss: 0.0759
  Sample: public class Calculator {
    private int value;
    
   ...
Epoch 100/100, Loss: 0.0706
  Sample: public static String reverse(String s) {
        StringBu...


---
## Step 5: Testing Generation

Now the trained model can generate Java code completions. The generation process:

1. Start with a prompt (partial code)
2. Feed it through the model
3. Sample the next character from the predicted distribution
4. Append to the sequence and repeat

### The Temperature Parameter

Temperature controls the randomness of generation:

- **temperature = 1.0**: Use the model's raw probabilities
- **temperature < 1.0**: "Sharper" distribution — more likely to pick the highest-probability token (more deterministic)
- **temperature > 1.0**: "Flatter" distribution — more uniform, more randomness (more creative/chaotic)

Mathematically, temperature divides the logits before softmax:
$$P_i = \frac{e^{z_i/T}}{\sum_j e^{z_j/T}}$$

As $T \to 0$, only the maximum logit gets probability 1 (argmax). As $T \to \infty$, the distribution becomes uniform.

In [53]:
def generate_completion(model, tokenizer, prompt: str, max_tokens: int = 100, temperature: float = 0.8):
    """Generate a completion for the given prompt."""
    model.eval()  # Switch to evaluation mode
    
    prompt_ids = torch.tensor([tokenizer.encode(prompt)]).to(device)
    generated = model.generate(prompt_ids, max_new_tokens=max_tokens, temperature=temperature)
    
    return tokenizer.decode(generated[0].tolist())


# Test with various prompts typical for Java code
test_prompts = [
    "public class ",
    "public static void ",
    "private int ",
    "return this.",
    "if (",
]

print("=" * 60)
print("GENERATION TESTING")
print("=" * 60)

for prompt in test_prompts:
    print(f"\nPrompt: '{prompt}'")
    completion = generate_completion(model, tokenizer, prompt, max_tokens=60)
    print(f"Result: {completion}")
    print("-" * 40)

GENERATION TESTING

Prompt: 'public class '
Result: public class Calculator {
    private int value;
    
    public Calculat
----------------------------------------

Prompt: 'public static void '
Result: public static void main(String[] args) {
        System.out.println("Hello, Wor
----------------------------------------

Prompt: 'private int '
Result: private int value;
    
    public Calculator() {
        this.value = 0
----------------------------------------

Prompt: 'return this.'
Result: return this.value;
    }
    
    public int getValue() {
        return
----------------------------------------

Prompt: 'if ('
Result: if (isEmpty(s)) {
            return s;
        }
        return
----------------------------------------


---
## Summary and Next Steps

### What Was Built

This notebook implemented a complete mini-GPT from scratch:

1. **Tokenization**: Converting characters to/from numerical indices
2. **Self-Attention**: The core mechanism that allows tokens to communicate
3. **Multi-Head Attention**: Parallel attention for different relationship types
4. **Feed-Forward Networks**: Non-linear transformations at each position
5. **Transformer Blocks**: Combining attention and FFN with residual connections
6. **Positional Embeddings**: Teaching the model about token order
7. **Training Loop**: Optimizing with cross-entropy loss
8. **Text Generation**: Autoregressive sampling with temperature control

### Current Limitations

The model was trained on a tiny dataset (~1KB of code). It should already:
- Learn basic Java syntax (braces, semicolons, indentation patterns)
- Memorize patterns from the training examples

But it won't generalize well to new patterns it hasn't seen.

### What Comes Next

To build a more capable model:

1. **More data**: Load a real dataset with thousands of Java files
2. **BPE tokenization**: Subword tokenization is more efficient for code than character-level
3. **Scale up**: Increase embed_dim, num_layers, num_heads, and context_length
4. **Proper evaluation**: Add train/validation split and track perplexity metrics
5. **Regularization**: Add dropout to prevent overfitting on larger models

### Experiments to Try

Before moving on, some experiments with this model:
- Change hyperparameters (embed_dim, num_layers, num_heads) and observe effects
- Add more Java code to SAMPLE_JAVA_CODE
- Try different temperature values during generation (0.5 vs 1.0 vs 1.5)
- Visualize attention patterns (which tokens attend to which?)